# Apple Quality Dataset

#1 
-IMPORT ALL REQUIRED LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn import feature_selection
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

import template


#2 
-READ FILES USING PANDAS PACKAGE

In [ ]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir,"Datasets","apple_quality.csv")
data = pd.read_csv(data_dir)

#3
-EXPLORATORY DATA ANALYSIS

In [ ]:
data.head()

In [ ]:
data.dtypes

As you can see that data typr mentioned here for **Acidity** is Object but its actually float so we need to conver it into float

In [ ]:
#data["Acidity"] = (data["Acidity"]).astype("float64")

In [ ]:
data.tail()
# now if you obseve at end there is string at end of the row 
# so remove it and rerun the above cell

In [ ]:
data  = data.drop(data.index[-1])

In [ ]:
data["Acidity"] = (data["Acidity"]).astype("float64")

In [ ]:
data.tail()

-Set **A_id** as index or remove it as it's not a feature

In [ ]:
data.set_index("A_id",inplace=True)

using
- describe()
- info()<br>

to get information regarding each column

In [ ]:
data.describe()

In [ ]:
data.info()

Here you can observe from above that there are no missing values in the data

In [ ]:
data_val = template.data_val(data)
column_types = data_val.get_column_type()
data_val.zero_count(column_types)

Also there are no zeros in the data cause sometimes erp or iot systems or legacy systems assigned zeros to the data sample where its null

In [ ]:
zeros_count = data_val.zero_count1()
print(zeros_count)

In [ ]:
schema = data_val.schema()
print(schema)

In [ ]:
sns.set_theme()

In [ ]:
sns.boxplot(data=data)
plt.tight_layout()

In [ ]:
for cat in pd.unique(data["Quality"]):
    print(f"{cat} =",(data["Quality"]==cat).sum())

In [ ]:
#data_scaled = data_val.scale_dataset(Oversample=True)

In [ ]:
#sns.boxplot(data=data_scaled)
plt.tight_layout()

In [ ]:
data_cleaned = data_val.remove_outliers()

In [ ]:
sns.boxplot(data_cleaned)
plt.tight_layout()

In [ ]:
data_scaled = data_val.scale_dataset()
sns.boxplot(data_scaled)

In [ ]:
sns.kdeplot(data=(data_scaled),)

In [ ]:

data_val.histplot_columns(2,4,figuresize = (15,15))

In [ ]:
encoder = OneHotEncoder(drop="first")

In [ ]:
for col,col_types in column_types.items():
    if col_types == "categorical":
        x= data[col].values.reshape(-1,1)
        encoder.fit(x)
        data[col]=encoder.transform(x).toarray()


In [ ]:
for col,col_types in column_types.items():
    if col_types == "categorical":
        x= data_scaled[col].values.reshape(-1,1)
        encoder.fit(x)
        data_scaled[col]=encoder.transform(x).toarray()



In [ ]:
data_scaled.head()

In [ ]:
sns.pairplot(data_scaled,hue="Quality")

# Feature Selection

In [ ]:
corrmap = data_scaled.corr()
sns.set_theme(style="white")
plt.figure(figsize=(11,9))
mask = np.triu(np.ones_like(corrmap, dtype=bool))
cmap = sns.diverging_palette(15,260, as_cmap=True)
sns.heatmap(corrmap,cmap=cmap,mask=mask,annot=True)


In [ ]:
x = data_scaled[data_scaled.columns[:-1]]
y = data_scaled[data_scaled.columns[-1]]

In [ ]:
x_train1,x_test1,y_train1,y_test1 = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
y_train1.head()

In [ ]:
x_train1.head()
print(x_train1.var(axis=0))

In [ ]:
sns.barplot(x_train1.var(axis=0))

In [ ]:

x_new = feature_selection.SelectKBest(feature_selection.f_classif,k=5).fit(x_train1,y_train1)

In [ ]:
selected_feature = x_new.get_feature_names_out()


In [ ]:
x_train2 = x_train1[selected_feature]
x_train2.head()

In [ ]:
y_train1.head()

In [ ]:
data1 = pd.concat([x_train2, y_train1],axis=1)
data1.head()

In [ ]:
sns.pairplot(data1,hue="Quality")

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(x_train1, y_train1)

print(f"Optimal number of features: {rfecv.n_features_}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming rfecv is your RFECV object
cv_results = pd.DataFrame(rfecv.cv_results_)

# Adding the n_features column
cv_results['n_features'] = range(1, len(cv_results) + 1)

# Plotting the results
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test accuracy")
plt.errorbar(
    x=cv_results["n_features"],
    y=cv_results["mean_test_score"],
    yerr=cv_results["std_test_score"],
    fmt='-o'  # Add this for better visualization (line with circle markers)
)
plt.title("Recursive Feature Elimination \nwith correlated features")
plt.show()


In [ ]:
import pandas as pd

# Assuming rfecv is your RFECV object
cv_results = pd.DataFrame(rfecv.cv_results_)
print(cv_results.columns)


In [ ]:
print(cv_results.head())


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import  BaggingClassifier


In [ ]:
from sklearn.metrics import precision_score,recall_score,classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score

In [ ]:
x_test1 = x_test1[selected_feature]

Given Dataset is of small size and does not occupy large memory based on that aplicable algorithms for classification are : 
1. Logistic Regression
2. K-nn classification
3. SVM classification
4. Decision tree
5. RandomForest
6. Gradient Boosting Machines
6. Adaboost
7. XGboost
8. LightGbm
9. CatBoost
10. Naive Bayes
11. SGD
12. Ridge Classifier
13. Bagging Classifier

In [ ]:
models = {
    "logistic_regression" : LogisticRegression(),
    "SVC_kernel" : SVC(kernel="rbf",gamma="scale"),
    "SVC_kernel_poly" : SVC(kernel="poly",gamma="scale",degree=3,coef0=1),
    "SVC_kernel_sigmoid" : SVC(kernel="sigmoid",gamma="scale",coef0= 1),
    "KNN_classification" : KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "SGDClassifier": SGDClassifier(),
    "RidgeClassifier": RidgeClassifier(),
    "BaggingClassifier": BaggingClassifier()
}
Accuracy = {}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train2,y_train1)

    y_train_pred = model.predict(x_train2)
    y_test_pred = model.predict(x_test1)

    print(f"\033[1m{i+1}. {list(models.keys())[i]}\033[0m")
    print(f"Training Accuracy : {model.score(x_train2,y_train1)}")
    print(f"Testing Accuracy : {model.score(x_test1,y_test1)}")
    print(f"Training Confusion Matrix : \n{confusion_matrix(y_train1,y_train_pred)}")
    print(f"Testing Confusion Matrix : \n{confusion_matrix(y_test1,y_test_pred)}")
    print(f"Training Classification Report :\n {classification_report(y_train1,y_train_pred)}")
    print("----------------------------------------------------------------")
    print(f"Testing Classification Report : \n{classification_report(y_test1,y_test_pred)}")
    print("----------------------------------------------------------------")
    print(f"Precision Report : {accuracy_score(y_train1,y_train_pred)}")
    print("================================================================")

    Accuracy[list(models.keys())[i]] = accuracy_score(y_test1,y_test_pred)

for i in Accuracy:
    print(f"{i} : {Accuracy[i]}")   

    




               

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, mean_squared_error

def select_models(X, y, interpretability='moderate'):
    # Determine if the problem is classification or regression
    if y.nunique() <= 10:
        problem_type = 'classification'
    else:
        problem_type = 'regression'

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    models = []
    if problem_type == 'classification':
        models = [
            ('Logistic Regression', LogisticRegression(max_iter=1000)),
            ('Decision Tree', DecisionTreeClassifier()),
            ('Random Forest', RandomForestClassifier()),
            ('Gradient Boosting', GradientBoostingClassifier()),
            ('SVM', SVC()),
            ('KNN', KNeighborsClassifier()),
            ('Naive Bayes', GaussianNB())
        ]
    else:
        models = [
            ('Linear Regression', LinearRegression()),
            ('Ridge Regression', Ridge()),
            ('Lasso Regression', Lasso()),
            ('Elastic Net Regression', ElasticNet()),
            ('Decision Tree Regressor', DecisionTreeRegressor()),
            ('Random Forest Regressor', RandomForestRegressor()),
            ('Gradient Boosting Regressor', GradientBoostingRegressor()),
            ('SVR', SVR()),
            ('KNN Regressor', KNeighborsRegressor())
        ]
    
    selected_models = []

    for name, model in models:
        model.fit(X_train, y_train)
        if problem_type == 'classification':
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f'{name} Accuracy: {accuracy}')
        else:
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            print(f'{name} Mean Squared Error: {mse}')

        # Select models based on interpretability
        if interpretability == 'high' and name in ['Logistic Regression', 'Linear Regression', 'Decision Tree']:
            selected_models.append(model)
        elif interpretability == 'moderate' and name in ['Random Forest', 'Gradient Boosting', 'Ridge Regression', 'Lasso Regression', 'Elastic Net Regression']:
            selected_models.append(model)
        elif interpretability == 'low' and name in ['Neural Networks', 'SVM', 'SVR']:
            selected_models.append(model)
    
    return selected_models



In [ ]:
print(len(x_train2))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, mean_squared_error

def select_model(X, y, interpretability='moderate'):
    # Determine problem type
    problem_type = 'classification' if len(y.unique()) <= 10 else 'regression'
    
    # Determine dataset size
    dataset_size = len(X)
    
    # Determine dataset dimensionality
    dataset_dimensionality = X.shape[1]
    
    # Select appropriate models based on dataset size and dimensionality
    if dataset_size <= 1000:
        if dataset_dimensionality <= 50:
            if interpretability == 'high':
                return LogisticRegression(), LinearRegression(), DecisionTreeClassifier()
            elif interpretability == 'moderate':
                return RandomForestClassifier(), GradientBoostingClassifier(), Ridge(), Lasso()
            elif interpretability == 'low':
                return SVC(), KNeighborsClassifier()
        else:
            if interpretability == 'high':
                return LogisticRegression(), LinearRegression(), DecisionTreeClassifier()
            elif interpretability == 'moderate':
                return RandomForestClassifier(), GradientBoostingClassifier(), Ridge(), Lasso()
            elif interpretability == 'low':
                return SVC(), KNeighborsClassifier()
    else:
        if interpretability == 'high':
            return DecisionTreeClassifier()
        elif interpretability == 'moderate':
            return RandomForestClassifier(), GradientBoostingClassifier()
        elif interpretability == 'low':
            return SVC()



In [ ]:
select_models(x_train2,y_train1)

In [ ]:
select_model(x_train2,y_train1)